In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import simplenn as sn

# Modelo `Sequential` para Redes Neuronales


Ya hemos implementado capas/modelos de todo tipo: densas, funciones de activación, de error, etc. Además, tenemos inicializadores, un optimizador basado en descenso de gradiente estocástico, y modelos que combinan otras capas como `LinearRegression` y `LogisticRegression`. 

Para dar el siguiente paso y poder definir redes neuronales simples, vamos a implementar el modelo `Sequential`. Este modelo generaliza las ideas aplicadas en `LinearRegression`, `LogisticRegression` y `Dense`, es decir, crear una capa en base a otras. En los casos anteriores, las capas a utilizar estaban predefinidas. `Sequential` nos permitirá utilizar cualquier combinación de capas que querramos.



# Creación de un modelo `Sequential`


Un modelo `Sequential` debe crearse con una lista de otros modelos/capas. De esta manera, específicaremos qué transformaciones y en qué orden se realizarán para obtener la salida de la red.

Podemos ver varios ejemplos en donde creamos un modelo de regresión lineal, logística, o una capa Dense en base al modelo `Sequential`.

`Sequential` también tiene un método muy útil, `summary()`, que nos permite obtener una descripción de las capas y sus parámetros.


In [ ]:
din=5
dout=3

# Creamos un modelo de regresión lineal 
layers = [sn.Linear(din,dout), sn.Bias(dout)]
linear_regression = sn.Sequential(layers,name="Regresión Lineal")
print(linear_regression.summary())


# Creamos un modelo de regresión lineal, pero sin la variable auxiliar `layers`
linear_regression = sn.Sequential([sn.Linear(din,dout),
                       sn.Bias(dout),
                      ],name="Regresión Lineal")
print(linear_regression.summary())

# Creamos un modelo de regresión logística 
logistic_regression = sn.Sequential([sn.Linear(din,dout),
                       sn.Bias(dout),
                       sn.Softmax(dout)
                      ],name="Regresión Logística")
print(logistic_regression.summary())


# Creamos un modelo tipo capa Dense con activación ReLU
dense_relu = sn.Sequential([sn.Linear(din,dout),
                       sn.Bias(dout),
                       sn.ReLU(dout)
                      ],name="Capa tipo Dense con activación ReLU")
print(dense_relu.summary())




# Redes de varias capas con `Sequential`

También vamos a crear nuestras primeras redes neuronales de varias capas, simplemente agregando más capas al modelo.

In [ ]:

# Creamos una red con dos capas Dense, y una dimensionalidad de 3 interna
network_layer2 = sn.Sequential([sn.Dense(din,3,"relu"),
                               sn.Dense(3,dout,"id")
                      ],name="Red de dos capas")
print(network_layer2.summary())



# Creamos una red con 4 capas Dense
# dimensiones internas de 2, 4 y 3
# y función de activación final softmax
network_layer4 = sn.Sequential([sn.Dense(din,2,"relu"),
                               sn.Dense(2,4,"tanh"),
                               sn.Dense(4,3,"sigmoid"),
                               sn.Dense(3,dout,"softmax"),
                      ],name="Red de dos capas")
print(network_layer4.summary())

# Paramétros de `Sequential`

El modelo `Sequential`  también permite obtener fácilmente los parámetros de todos sus modelos internos. Para eso ya hemos implementado el método `get_parameters` que permite obtener _todos_ los parámetros de los modelos internos, pero renombrados para que si, por ejemplo, dos modelos tienen el mismo nombre de sus parámetros, estos nombres no se repitan.


In [ ]:
print("Nombres de los parámetros de network_layer2")
print(network_layer2.get_parameters().keys())

print("Nombres de los parámetros de network_layer4")
print(network_layer4.get_parameters().keys())

# Método `forward` de `Sequential`


Vamos ahora a implementar el método `forward` de `Sequential`. Para eso, dada una entrada `x`, y una sucesión de modelos `M_1,M_2,...,M_n` de `Sequential`, debemos calcular la salida `y` como:

$$ y = M_n(...(M_2(M_1(x))...)$$

En términos de código, debemos iterar por los posibles modelos (empezando por el primero) y aplicar el método `forward`

````python
for m in models:
    x = m.forward(x)
return x
````

Implementá `forward` para la clase `Sequential` en `simplenn/models/sequential.py`. 

In [ ]:
x = np.array([[3,-7],
             [-3,7]])

w = np.array([[2, 3, 4],[4,5,6]])
b = np.array([1,2,3])
linear_initializer = sn.initializers.Constant(w)
bias_initializer = sn.initializers.Constant(b)
layer=sn.Sequential([sn.Linear(2,3,initializer=linear_initializer),
                     sn.Bias(3,initializer=bias_initializer)
                    ])
y = np.array([[-21, -24, -27],
              [ 23, 28,  33]])

sn.utils.check_same(y,layer.forward(x))

linear_initializer = sn.initializers.Constant(-w)
bias_initializer = sn.initializers.Constant(-b)
layer=sn.Sequential([sn.Linear(2,3,initializer=linear_initializer),
                     sn.Bias(3,initializer=bias_initializer)
                    ])
sn.utils.check_same(-y,layer.forward(x))

# Método `backward`


Al igual que con `Dense`, para implementar el `backward`, también deberás llamar al `backward` de cada uno de los modelos en el orden _inverso_ al del forward. Dado un tensor `δEδy` que contiene las derivadas del error respecto a cada valor de la salida `y`, debemos calcular:
* `δEδx`, la derivada del error respecto a la entrada `x`
* `δEδp_i`, la derivada del error respecto a cada parámetro `p_i`

Para ello, debemos iterar por los posibles modelos (empezando por el último) y aplicar el método `backward`, propagando el error para atrás, y recolectando en el proceso lo más importante, que son las derivadas del error respecto a los parámetros. En términos de código,

````python
δEδp = {}
for m_i in reverse(models):
    δEδy, δEδp_i = m_i.backward(δEδy)
    agregar los gradientes de δEδp_i a δEδp
return δEδy,δEδp
````
En este caso, también te ayudamos con la función `merge_gradients` que podés llamar como `self.merge_gradients(layer,δEδp,gradients)`. Esta función te permite agregar los parámetros `δEδp` de la capa `layer` al diccionario de gradientes final `gradients` que se debe retornar.


In [ ]:
samples = 100
batch_size=2
features_in=3
features_out=5
input_shape=(batch_size,features_in)

# Test derivatives of a Sequential model with random values for `w`
layer=sn.Sequential([sn.Linear(features_in,features_out),
                     sn.Bias(features_out),
                     sn.ReLU()
                    ])
sn.utils.check_gradient.common_layer(layer,input_shape,samples=samples)    


# !Felicitaciones! 

!Implementaste todas las funciones básicas de una librería de redes neuronales!

Ahora vamos a definir algunas redes neuronales para mejorar el desempeño respecto de los modelos lineales (Regresión Lineal y Regresión Logística)
